# Clasificador regresion lineal - hito 3

In [274]:
import pandas as pd
import numpy as np
import pickle

import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme()
sns.set_theme(style="whitegrid")
plt.rc('axes', titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rcParams.update({'font.size': 16})
plt.rcParams['axes.titlesize'] = 16
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams.update({'lines.markeredgewidth': 1})
plt.rcParams.update({'errorbar.capsize': 2})

file_names = {
    "df_es_mapping": "../../Data/mapping/df_es_mapping.pickle",
    "df_us_mapping": "../../Data/mapping/df_us_mapping.pickle",
    
    "df_es_test": "../../Data/test/df_es_test.pickle",
    "df_us_test": "../../Data/test/df_us_test.pickle",
    
    "df_es_train": "../../Data/train/df_es_train.pickle",
    "df_us_train": "../../Data/train/df_us_train.pickle",
    
    "df_es_trial": "../../Data/trial/df_es_trial.pickle",
    "df_us_trial": "../../Data/trial/df_us_trial.pickle",
}

# mas imports
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()

**cargar sets**

In [275]:
df_es_train = pickle.load(open(file_names["df_es_train"], "rb"))
df_es_trial = pickle.load(open(file_names["df_es_trial"], "rb"))
df_es_test = pickle.load(open(file_names["df_es_test"], "rb"))

df_us_train = pickle.load(open(file_names["df_us_train"], "rb"))
df_us_trial = pickle.load(open(file_names["df_us_trial"], "rb"))
df_us_test = pickle.load(open(file_names["df_us_test"], "rb"))


In [276]:
# Transformamos el atributo multiclase categorico a variables binarias
df_us_train2 = pd.get_dummies(df_us_train , columns = ["label"])
df_us_trial2 = pd.get_dummies(df_us_trial , columns = ["label"])
df_us_test2 = pd.get_dummies(df_us_test , columns = ["label"])
#df_us_train2.head()
df_us_test2.head()

,id,text,label_0,label_1,label_10,label_11,label_12,label_13,label_14,label_15,...,label_18,label_19,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9
0,test0,en Pelham Parkway,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,test1,The calm before...... | w/ sofarsounds @user |...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,test2,Just witnessed the great solar eclipse @ Tampa...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,test3,This little lady is 26 weeks pregnant today! E...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,test4,"Great road trip views! @ Shartlesville, Pennsy...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**pre-procesamiento**

In [277]:
%%time
df_us_train2['tokenized_text'] = df_us_train2['text'].str.lower().apply(lambda x: " ".join(tt.tokenize(x)))
df_us_train2.head()

Wall time: 31.9 s


,id,text,label_0,label_1,label_10,label_11,label_12,label_13,label_14,label_15,...,label_19,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9,tokenized_text
0,729044324441186304,Selfies for summatime @ Drexel University,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,selfies for summatime @ drexel university
1,663834134037442560,Ready to be a bulldog with rasso #hailstate #i...,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,ready to be a bulldog with rasso #hailstate #i...
2,747449193350963200,#scored my new #matcotools #slidehammer weight...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,#scored my new #matcotools #slidehammer weight...
3,691439672761925637,@user last night was so much fun @ Skyway Thea...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,@user last night was so much fun @ skyway theatre
4,758118895618109440,love beach days @ Manasquan Beach,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,love beach days @ manasquan beach


In [278]:
df_us_test2['tokenized_text'] = df_us_test2['text'].str.lower().apply(lambda x: " ".join(tt.tokenize(x)))

In [279]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=10) # minima cantidad de veces que tiene que aparecer una palabra para ser considerado 
X_train_bow = vectorizer.fit_transform(df_us_train2["tokenized_text"])
X_test_bow = vectorizer.transform(df_us_test2["tokenized_text"])

In [280]:
# 
X_train_bow.shape[0]
#print(X_test_bow)

387292

## Regresión lineal en ingles

In [281]:
# Probamos para el label 0
%time
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train_bow, df_us_train2["label_17"])

#reg.predict(np.array([[3, 5]]))
#array([16.])

Wall time: 0 ns


LinearRegression()

In [282]:
reg.score(X_train_bow, df_us_train2["label_17"])

0.4337708335088303

In [283]:
coefs = reg.coef_

In [284]:
coefs.size

17777

In [285]:
from sklearn.metrics import classification_report
y_pred = reg.predict(X_test_bow)
print(y_pred)

[ 0.11338276  0.04996757  0.05058278 ...  0.0382701  -0.02656909
  0.01485125]


In [286]:
# y pred no es binario, hay que darle un cuttoff donde aceptemos que el algoritmo acerto, eligimos el 50 % 
# de esta forma podemos usar metricas tipicas
# c = corte

# Entonces si el algoritmo predice con un número entre 0 y 1 aprox, 
def cutoff(lista, c):
    i = 0
    listanueva = []
    for i in lista:
        if i > c:
            listanueva.append(1)
        else:
            listanueva.append(0)
            
    return listanueva       

In [287]:
df_us_mapping = pickle.load(open(file_names["df_us_mapping"], "rb"))
#df_us_mapping = df_us_mapping.sort_values('label')
print(df_us_mapping)

   label emoji                              name
0      0     ❤                       _red_heart_
1      1     😍     _smiling_face_with_hearteyes_
2      2     😂          _face_with_tears_of_joy_
3      3     💕                      _two_hearts_
4      4     🔥                            _fire_
5      5     😊  _smiling_face_with_smiling_eyes_
6      6     😎    _smiling_face_with_sunglasses_
7      7     ✨                        _sparkles_
8      8     💙                      _blue_heart_
9      9     😘             _face_blowing_a_kiss_
10    10     📷                          _camera_
11    11    🇺🇸                   _United_States_
12    12     ☀                             _sun_
13    13     💜                    _purple_heart_
14    14     😉                    _winking_face_
15    15     💯                  _hundred_points_
16    16     😁  _beaming_face_with_smiling_eyes_
17    17     🎄                  _Christmas_tree_
18    18     📸               _camera_with_flash_
19    19     😜      

In [288]:
from sklearn.metrics import classification_report

y_pred = reg.predict(X_test_bow)

Puntodecorte = 0.5 # punto del cuttoff
y_predcut = cutoff(y_pred, Puntodecorte)

print(classification_report(df_us_test2["label_17"], y_predcut,output_dict = True ))

{'0': {'precision': 0.9804392029280196, 'recall': 0.995108863894335, 'f1-score': 0.9877195677779484, 'support': 48455}, '1': {'precision': 0.7109756097560975, 'recall': 0.3773462783171521, 'f1-score': 0.49302325581395345, 'support': 1545}, 'accuracy': 0.97602, 'macro avg': {'precision': 0.8457074063420585, 'recall': 0.6862275711057435, 'f1-score': 0.740371411795951, 'support': 50000}, 'weighted avg': {'precision': 0.9721127778990071, 'recall': 0.97602, 'f1-score': 0.9724334517382609, 'support': 50000}}


In [289]:
%%time
#Ahora vemos para cada label
import numpy as np
from sklearn.metrics import precision_recall_fscore_support


def metrica_clase1(y_real,y_pred,emoji):
    Met_regresion_dict = classification_report(y_real, y_pred , output_dict = True )
    return print("|",df_us_mapping["emoji"][emoji],"|","Precisión =", Met_regresion_dict.get("1").get("precision"),"|", "Recall =" ,Met_regresion_dict.get("1").get("recall"),"|","f1-score =" ,Met_regresion_dict.get("1").get("f1-score"),"|",df_us_mapping["emoji"][emoji],"|")

Puntodecorte = 0.5 # punto del cuttoff
def metrica_clase1_todos(train_bow,df_label):
    Puntodecorte = 0.5
    i = 0
    f = df_label.shape[1]
    # el for pasa de i = 0 hasta el 17 maoma
    for i in range(f-3):
        ###
        reg = LinearRegression()
        reg.fit(train_bow, df_label[str("label_")+str(i)])
        y_predcut = cutoff(reg.predict(train_bow), Puntodecorte)
        print (metrica_clase1(df_label[str("label_")+str(i)],y_predcut, i))
        
metrica_clase1_todos( X_train_bow , df_us_train2)       
# AL UTILIZAR COMO PUNTO DE CORTE SOBRE EL 50% QUE SI
    

| ❤ | Precisión = 0.6235406602254429 | Recall = 0.1481982035856526 | f1-score = 0.2394789482330431 | ❤ |
None
| 😍 | Precisión = 0.680140597539543 | Recall = 0.03781697366492402 | f1-score = 0.07165008099976858 | 😍 |
None
| 😂 | Precisión = 0.7563370403767158 | Recall = 0.18687493811268444 | f1-score = 0.29970026003930367 | 😂 |
None
| 💕 | Precisión = 0.7111111111111111 | Recall = 0.006402881296583463 | f1-score = 0.0126914877794854 | 💕 |
None
| 🔥 | Precisión = 0.764804469273743 | Recall = 0.2732262249276519 | f1-score = 0.40261745459892645 | 🔥 |
None
| 😊 | Precisión = 0.8666666666666667 | Recall = 0.0028118747634239983 | f1-score = 0.005605562442731634 | 😊 |
None
| 😎 | Precisión = 0.7356948228882834 | Recall = 0.015764582238570676 | f1-score = 0.030867726077512286 | 😎 |
None
| ✨ | Precisión = 0.7250996015936255 | Recall = 0.039308855291576676 | f1-score = 0.07457488219627126 | ✨ |
None
| 💙 | Precisión = 0.7667560321715817 | Recall = 0.022587268993839837 | f1-score = 0.043881856540084384 

Se encuentran resultados mas altos de lo esperado en la precision, pero sin embargo el recall no es muy alto, el algoritmo de regresión lineal falla mucho en los falsos negativos ( Dice que no es el emoji, pero en realidad si es), a cambio de tener una precisión más alta. 

Para encontrar el top de palabras, vamos a predecir y luego este valor de predicción va a corresponder al valor que la regresión le asocio al coeficiente de cada regresión

In [290]:
%%time
reg = LinearRegression()
reg.fit(X_train_bow, df_us_train2["label_17"])
vocab_length = X_train_bow.shape[1]
proba_matrix = np.array([reg.predict(np.eye(1,vocab_length,k))[0] for k in range(vocab_length)])

Wall time: 9.9 s


In [291]:
def topPalabras(proba_matrix,emoji_id,k=5):
    # retorna las palabras para las cuales el emoji en cuestión tiene mas probabilidad
    prob = proba_matrix[:]  # mmm
    ind = np.argpartition(prob,-k)[-k:]
    val = prob[ind]
    palabras = [vectorizer.inverse_transform([np.eye(1,vocab_length,k)[0]])[0][0] for k in ind]
    return palabras, val

In [292]:
i = 17
map_emojis = df_us_mapping["label"].values
print(df_us_mapping["emoji"][int(map_emojis[i])])
topPalabras(proba_matrix,i)

🎄


(['ohchristmastree', 'holidaze', 'litmas', 'xmas2015', 'xmas2016'],
 array([0.54584545, 0.57313968, 0.62446983, 0.7050567 , 0.76373461]))

In [293]:
%%time
for i in range(20):
    reg = LinearRegression()
    reg.fit(X_train_bow, df_us_train2[str("label_")+str(i)])
    proba_matrix = np.array([reg.predict(np.eye(1,vocab_length,k))[0] for k in range(vocab_length)])
    print(df_us_mapping["emoji"][int(map_emojis[i])])
    pal, val = topPalabras(proba_matrix,i)
    print(dict([(pal[j],val[j]) for j in range(len(pal))]))

❤
{'sibabes': 0.804948529378079, 'snowwhite': 0.8203815604017426, '1luv': 0.9551082292107682, 'cityofbrotherlylove': 1.0276042069771996, 'kfodiaries': 0.9732962170496406}
😍
{'asada': 0.6821768010788973, 'gyu': 0.8567913547961459, 'enamorada': 0.764880037442025, 'eatgood': 0.8798387923167201, 'encanta': 0.6974620745801943}
😂
{'lmfaoooo': 0.7756739280963982, 'foolin': 0.7836170807026172, 'hysterical': 0.8031867993103163, 'postavideoyoucantexplain': 0.9092025087300106, 'clownin': 0.980526728422956}
💕
{'froomies': 0.4184682101946456, 'twinny': 0.5270839756126672, 'endorsement': 0.5009280482968419, 'honeybglam': 0.5480836741094768, 'bakers': 0.42597044968754366}
🔥
{'linkinmybio': 0.7315935943422897, 'fiya': 0.748510479195848, 'feeltheburn': 0.8444973901079282, 'instant_classic': 0.845625695616753, 'onfire': 0.9160725933546516}
😊
{'iah': 0.35770813114790895, 'allah': 0.3617374629968806, 'snowglobe': 0.38610150714220465, 'trigger': 0.46180553488716364, 'goodtime': 0.528529456916411}
😎
{'sungl

Finalmente se encontro otro clasificador que puede relacionar cada palabra frecuente de un idioma a una probabilidad, si bien ya se habian encontrado probabilidades asociadas a palabras mediante el clasificador MultinomialNB, ahora se obtiene resultados mediante los coeficientes de la regresión lineal que relacionan a cada palabra con una probabilidad, lo siguiente es repetir el codigo para español.

## Regresión lineal en español

In [294]:
# Transformamos el atributo multiclase categorico a variables binarias
df_es_train2 = pd.get_dummies(df_es_train , columns = ["label"])
df_es_trial2 = pd.get_dummies(df_es_trial , columns = ["label"])
df_es_test2 = pd.get_dummies(df_es_test , columns = ["label"])
#df_us_train2.head()
df_es_test2.head()

,id,text,label_0,label_1,label_10,label_11,label_12,label_13,label_14,label_15,...,label_17,label_18,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9
0,test0,Buenos días desde Valencia en Comunidad Valenc...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,test1,"Anoche en la #prefería con @user ,mi prima eva...",0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,test2,"Porfavor llevarlas a reciclar,necesitamos más ...",0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,test3,El vecino roquero que todos queremos tener en ...,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,test4,Es un placer contar con profesionales del sect...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [295]:
%%time
df_es_train2['tokenized_text'] = df_es_train2['text'].str.lower().apply(lambda x: " ".join(tt.tokenize(x)))

Wall time: 5.11 s


In [296]:
df_es_test2['tokenized_text'] = df_es_test2['text'].str.lower().apply(lambda x: " ".join(tt.tokenize(x)))

In [297]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=10)   # Minimo de 10 ocurrencias para considerar una palabra
X_train_bow = vectorizer.fit_transform(df_es_train2["tokenized_text"])
X_test_bow = vectorizer.transform(df_es_test2["tokenized_text"])

In [298]:
df_es_mapping = pickle.load(open(file_names["df_es_mapping"], "rb"))
df_es_mapping

,label,emoji,name
0,0,❤,_red_heart_
1,1,😍,_smiling_face_with_hearteyes_
2,2,😂,_face_with_tears_of_joy_
3,3,💕,_two_hearts_
4,4,😊,_smiling_face_with_smiling_eyes_
5,5,😘,_face_blowing_a_kiss_
6,6,💪,_flexed_biceps_
7,7,😉,_winking_face_
8,8,👌,_OK_hand_
9,9,🇪🇸,_Spain_


In [299]:
%%time
#Ahora vemos para cada label
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

def metrica_clase1(y_real,y_pred,emoji):
    Met_regresion_dict = classification_report(y_real, y_pred , output_dict = True )
    return print("|",df_es_mapping["emoji"][emoji],"|","Precisión =", Met_regresion_dict.get("1").get("precision"),"|", "Recall =" ,Met_regresion_dict.get("1").get("recall"),"|","f1-score =" ,Met_regresion_dict.get("1").get("f1-score"),"|",df_es_mapping["emoji"][emoji],"|")

Puntodecorte = 0.5 # punto del cuttoff
def metrica_clase1_todos(train_bow,df_label):
    Puntodecorte = 0.5
    i = 0
    f = df_label.shape[1]
    # el for pasa de i = 0 hasta el 17 maoma
    for i in range(f-3):
        ###
        reg = LinearRegression()
        reg.fit(train_bow, df_label[str("label_")+str(i)])
        y_predcut = cutoff(reg.predict(train_bow), Puntodecorte)
        print (metrica_clase1(df_label[str("label_")+str(i)],y_predcut, i))
        
metrica_clase1_todos( X_train_bow , df_es_train2)       
# AL UTILIZAR COMO PUNTO DE CORTE SOBRE EL 50% QUE SI
    

| ❤ | Precisión = 0.640560593569662 | Recall = 0.09650975034157247 | f1-score = 0.1677461139896373 | ❤ |
None
| 😍 | Precisión = 0.6945454545454546 | Recall = 0.08355936652375535 | f1-score = 0.14917213370821616 | 😍 |
None
| 😂 | Precisión = 0.7568710359408034 | Recall = 0.13902912621359223 | f1-score = 0.23490813648293962 | 😂 |
None
| 💕 | Precisión = 0.7674418604651163 | Recall = 0.0061705310396409875 | f1-score = 0.012242626599888704 | 💕 |
None
| 😊 | Precisión = 0.8203125 | Recall = 0.019273127753303965 | f1-score = 0.03766140602582496 | 😊 |
None
| 😘 | Precisión = 0.8571428571428571 | Recall = 0.013114754098360656 | f1-score = 0.025834230355220672 | 😘 |
None
| 💪 | Precisión = 0.7531305903398927 | Recall = 0.13476312419974393 | f1-score = 0.2286179744773283 | 💪 |
None
| 😉 | Precisión = 0.8974358974358975 | Recall = 0.01122874558870709 | f1-score = 0.02217997465145754 | 😉 |
None
| 👌 | Precisión = 0.75 | Recall = 0.0020804438280166435 | f1-score = 0.004149377593360996 | 👌 |
None
| 🇪🇸 | Pr

D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


| 💘 | Precisión = 0.0 | Recall = 0.0 | f1-score = 0.0 | 💘 |
None
| 😁 | Precisión = 1.0 | Recall = 0.0027235587834770767 | f1-score = 0.005432322317790855 | 😁 |
None
Wall time: 22.1 s


In [300]:
%%time

vocab_length = X_train_bow.shape[1]
#proba_matrix = np.array([reg.predict(np.eye(1,vocab_length,k))[0] for k in range(vocab_length)])

for i in range(19):
    reg = LinearRegression()
    reg.fit(X_train_bow, df_es_train2[str("label_")+str(i)])
    proba_matrix = np.array([reg.predict(np.eye(1,vocab_length,k))[0] for k in range(vocab_length)])
    print(df_es_mapping["emoji"][int(map_emojis[i])])
    pal, val = topPalabras(proba_matrix,i)
    print(dict([(pal[j],val[j]) for j in range(len(pal))]))

❤
{'brindemos': 0.6171623483187451, 'fam': 0.6346323644888661, 'bachiller': 0.6495989749790456, 'valentín': 0.8008964010668764, 'ilovemadrid': 0.9422002659827386}
😍
{'felizañonuevo': 0.5784216021899539, 'familla': 0.5786858410584692, 'hermosos': 0.5849570548723055, 'james': 0.6252880697720704, 'preciosidad': 0.6134790639393111}
😂
{'jajajajajajajajaja': 0.7001654944377486, 'parto': 0.7445694185317469, 'aburrimiento': 0.774246266599665, 'meo': 0.8284886166645705, 'caretos': 0.7536431919924791}
💕
{'apuesto': 0.39092182611856946, 'imprescindibles': 0.3998922255897438, 'envío': 0.6174197792164663, 'vicálvaro': 0.4267764270658278, 'xeraco': 0.43886388576990565}
😊
{'invitación': 0.5004163027279447, 'perdiz': 0.5019288613362131, 'hashtags': 0.5279884467789383, 'comprando': 0.5787633982969738, 'esmorzant': 0.8017134414691671}
😘
{'besitos': 0.3598949919871568, 'ifach': 0.37693235544088693, 'besazo': 0.4808408450328213, 'rosita': 0.49784033978370773, 'besito': 0.49904464855369973}
💪
{'pretemporad

Finalmente se obtiene las palabras que son mas frecuentes según que emoji en español.